In [ ]:
#!conda create --name pem python=3.10 -y
#!conda activate pem

#!conda install pytorch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0 cudatoolkit=11.3 -c pytorch
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

!git clone https://github.com/NiccoloCavagnero/PEM.git
import os

# Ottieni il percorso corrente
current_path = os.getcwd()
print(current_path)

# Cambia directory
os.chdir('PEM')

# Ottieni il nuovo percorso
current_path = os.getcwd()
print(current_path)

!pip install -r requirements.txt

Scarico il dataset

In [ ]:
import os
# Ottieni il percorso corrente
current_path = os.getcwd()
print(current_path)

# Cambia directory
os.chdir('datasets')

# Ottieni il nuovo percorso
current_path = os.getcwd()
print(current_path)

In [ ]:

import zipfile
import subprocess

# Funzione per scaricare un file e decomprimerlo
def download_and_extract(url, zip_path, extract_path):
    # Se il file ZIP esiste già, non scaricarlo di nuovo
    if not os.path.exists(zip_path):
        print(f"Downloading {url} to {zip_path}...")
        # Scarica il file usando curl
        try:
            subprocess.run(["curl", "-L", "-o", zip_path, url], check=True)
        except subprocess.CalledProcessError as e:
            print(f"Error downloading file: {e}")
            return

    # Crea la directory di estrazione se non esiste
    os.makedirs(extract_path, exist_ok=True)

    # Estrai il file ZIP
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Extracted files to {extract_path}")
    except zipfile.BadZipFile as e:
        print(f"Error extracting {zip_path}: {e}")
        return

    # Elenca i file estratti
    extracted_files = os.listdir(extract_path)
    print(f"Extracted files in {extract_path}:", extracted_files)

# Percorsi principali
base_dir = os.path.expanduser("loveDa_zipped")

# Crea la directory base se non esiste
os.makedirs(base_dir, exist_ok=True)

# Dataset Val
val_url = "https://zenodo.org/records/5706578/files/Val.zip?download=1"
val_zip_path = os.path.join(base_dir, "Val.zip")
val_extract_path = os.path.join(base_dir, "")
download_and_extract(val_url, val_zip_path, val_extract_path)

# Dataset Train
train_url = "https://zenodo.org/records/5706578/files/Train.zip?download=1"
train_zip_path = os.path.join(base_dir, "Train.zip")
train_extract_path = os.path.join(base_dir, "")
download_and_extract(train_url, train_zip_path, train_extract_path)

Formatto il dataset come richiesto
LOVEDA
├───Test
│   ├───Rural
│   │   └───images_png
│   └───Urban
│       └───images_png
├───Train
│   ├───Rural
│   │   ├───images_png
│   │   └───masks_png
│   └───Urban
│       ├───images_png
│       └───masks_png
└───Val
    ├───Rural
    │   ├───images_png
    │   └───masks_png
    └───Urban
        ├───images_png
        └───masks_png

in
loveDa
    ├───train
    │   ├───annotations // mask contenute in Train/Urban/mask_png
    │   └───images // immagini contenute in Train/Urban/images_png
    ├───val
    │   ├───annotations  // mask contenute in Val/Urban/mask_png
    │   └───images  // immagini contenute in Val/Urban/images_png
    └───val_rural
        ├───annotations  // mask contenute in Val/Rural/mask_png
        └───images   // immagini contenute in Val/Rural/images_png

In [6]:
import os
import shutil
from pathlib import Path

def reorganize_loveda(source_dir, target_dir):
    # Crea le directory principali
    splits = {
        'train': ['Urban'],
        'val': ['Urban'],
        'val_rural': ['Rural']
    }

    for split, areas in splits.items():
        # Crea le sottodirectory per ogni split
        (Path(target_dir) / split / 'images').mkdir(parents=True, exist_ok=True)
        (Path(target_dir) / split / 'annotations').mkdir(parents=True, exist_ok=True)

        for area in areas:
            # Source paths
            if split == 'val_rural':
                src_split = 'Val'
            elif split == 'val':
                src_split = 'Val'
            else:
                src_split = 'Train'

            src_img = Path(source_dir) / src_split / area / 'images_png'
            src_mask = Path(source_dir) / src_split / area / 'masks_png'

            # Target paths
            dst_img = Path(target_dir) / split / 'images'
            dst_mask = Path(target_dir) / split / 'annotations'

            # Copia le immagini
            if src_img.exists():
                for img in src_img.glob('*'):
                    shutil.copy2(img, dst_img / img.name)

            # Copia le maschere se esistono
            if src_mask.exists():
                for mask in src_mask.glob('*'):
                    shutil.copy2(mask, dst_mask / mask.name)

    # Elimina la directory sorgente dopo la copia
    shutil.rmtree(source_dir)

if __name__ == "__main__":
    source_directory = "loveDa_zipped"  # Directory sorgente
    target_directory = "loveDa"  # Directory destinazione

    reorganize_loveda(source_directory, target_directory)


In [ ]:

# Ottieni il percorso corrente
current_path = os.getcwd()
print(current_path)

# Cambia directory
os.chdir('..')

# Ottieni il nuovo percorso
current_path = os.getcwd()
print(current_path)

inserisci le cartelle in config, lovaDa.py in dataset e  
# semantic segmentation
        if evaluator_type in ["sem_seg", "ade20k_panoptic_seg", "loveda_seg"]:
            evaluator_list.append(
                SemSegEvaluator(
                    dataset_name,
                    distributed=True,
                    output_dir=output_folder,
                )
            )

in train_net.py

In [ ]:
import os
import gdown

# Ottieni il percorso corrente
current_path = os.getcwd()
print(current_path)


# Crea la directory per i modelli pre-addestrati
os.makedirs('pretrained_models', exist_ok=True)
os.chdir('pretrained_models')

# Download dei modelli usando gdown

!gdown 1DFoXcV42zy-apUcMh5P8WhsXMRJofgl8
!gdown 1Y5belNkq3Dn-EYgSKY-ICiPsN4TZXoXO

# Converti i modelli
!python ../tools/convert-pretrained-stdc-model-to-d2.py STDCNet813M_73.91.tar STDC1.pkl
!python ../tools/convert-pretrained-stdc-model-to-d2.py STDCNet1446_76.47.tar STDC2.pkl

# Torna alla directory principale
os.chdir('..')


In [ ]:

import torch
import gc

# Clear cache
torch.cuda.empty_cache()
# Collect garbage
gc.collect()
# Check allocated memory
print(torch.cuda.memory_allocated() / 1024**2, "MB")
# Check cached memory
print(torch.cuda.memory_cached() / 1024**2, "MB")


!python train_net.py \
       --config-file ./configs/loveDa/semantic-segmentation/pem_R50_bs32_90k.yaml \
       --num-gpus 1 \
       DATALOADER.NUM_WORKERS 0